# 概要
* 1_data_preparationで実行した、S3にアップロードした音声ファイルに対してtranscribeで文字起こしを行う

In [ ]:
import boto3
import pandas as pd
from time import sleep
client = boto3.client('transcribe')

In [ ]:
# 定数
# 音声ファイル格納バケット（INPUT)と、文字起こし結果出力バケット（OUTPUT)
INPUT_BUCKET_NAME = 'transcribe-input-bucket-202004161130'
OUTPUT_BACKET_NAME = 'transcribe-output-bucket-202004161130'

In [ ]:
# resume 用の処理管理変数
# statusがTrueならジョブを実行させたことになる
file_list = list(pd.read_csv('./label.csv')['file_name'].values)
status = [False] * len(file_list)

In [ ]:
input_bucket = 's3://' + INPUT_BUCKET_NAME + '/'
output_bucket = OUTPUT_BACKET_NAME

In [ ]:
# 途中で失敗したときの resume 用処理
file_list = file_list[status.index(False):]

for i,file in enumerate(file_list):
    # ジョブ名は使いまわしのため、すでにある場合は削除する
    # get_transcription_jobはjob名が存在しない場合はエラーを吐くのでtry-exceptを利用
    try:
        job_name = client.get_transcription_job(
            TranscriptionJobName = file
        )['TranscriptionJob']['TranscriptionJobName']
        client.delete_transcription_job(TranscriptionJobName=job_name)
    except:
        pass
    # transcription_job
    response = client.start_transcription_job(
        TranscriptionJobName=file,
        Media={
            'MediaFileUri':input_bucket + file
        },
        MediaFormat='wav',
        LanguageCode='ja-JP',
        OutputBucketName= output_bucket
    )
    status[i] = True
    # APIを20回たたくごとに40秒のsleepを入れる
    # APIはサービスクオータが、緩和無しだとAPI同時実行数100で制限しているため、1ジョブ60分として、なんとなく20回で30秒のsleep
    if i%20==19:
        print(i)
        sleep(40)